In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Familiar stack packages
import numpy as np
from lsst.daf.butler import Butler
import lsst.geom as geom
from astropy.io import ascii as astascii
import glob
import anacal
from astropy.visualization import ZScaleInterval
import astropy.table as astTable
import fitsio
import pandas as pd

import astropy.units as u
from astropy.cosmology import Planck18 as cosmo
import gc

In [4]:
# CAMIRA
if False:
    # Path to your input .dat file
    input_file = "/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/s23b-camira/camira_s23b_wide_v2.dat"
    # Define the column names (since your file doesn't have a header line)
    column_names = ["ra", "dec", "z_cl", "N_mem", "logMs", "z_bcg", "ID"]
    # Read the .dat file assuming whitespace-separated values
    df = pd.read_csv(input_file, delim_whitespace=True, names=column_names)
    df["ID"] = df["ID"].astype(int)
    # Save to a CSV file
    df.to_csv("/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/s23b-camira/camira_s23b_wide_v2.csv", index=False)

In [5]:
data = astTable.Table.read("/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/s23b-camira/camira_s23b_wide_v2.csv")
print(data["dec"].min())
print(data["dec"].max())
data = data[(data["z_cl"] < 0.25) & (data["z_bcg"] < 0.25)]
print(len(data))

-6.687811
53.099959
2477


In [6]:
data2 = astTable.Table.read("/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/spec-galaxy-zlt03.fits")
data2 = data2[(data2["redshift"] < 0.21) & (data2["mag_i"] < 18)]

In [7]:
obs_repo = '/lustre/work/xiangchong.li/work/hsc_s23b_sim'
obs_collection = 'version1/image'

obs_butler = Butler(obs_repo, collections=obs_collection)
obs_registry = obs_butler.registry
skymap = obs_butler.get('skyMap', skymap="hsc")

full = fitsio.read("/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/tracts_fdfc_v1_trim2.fits")
tract_list = np.unique(full["tract"])
db_dir = "/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/"

In [ ]:
# Example array of physical sizes (in Mpc)
du = data2
for i, entry in enumerate(full):
    tract_id = entry["tract"]
    patch_db = entry["patch"]
    patch_x = patch_db // 100
    patch_y = patch_db % 100
    patch_id = patch_x + patch_y * 9
    
    db_dir = "/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/"
    outdir = f"{db_dir}/s23b-brightGalaxyMask/tracts/{tract_id}/{patch_id}"
    os.makedirs(outdir, exist_ok=True)
    out_fname = os.path.join(outdir, "catalog.fits")
    if os.path.isfile(out_fname):
        continue
    patch_info = skymap[tract_id][patch_id]
    wcs = patch_info.getWcs()
    bbox = patch_info.getOuterBBox()
    poly_p = patch_info.getOuterSkyPolygon().getBoundingCircle()
    msk = poly_p.contains(du["ra"] / 180 * np.pi, du["dec"]/ 180 * np.pi)
    dd =  du[msk]
    if len(dd) > 0:
        x, y = wcs.skyToPixelArray(ra=dd["ra"], dec=dd["dec"], degrees=True)
        x = np.array(x - bbox.getBeginX(), dtype=float)
        y = np.array(y - bbox.getBeginY(), dtype=float)
        
        # Angular diameter distance at z
        DA = cosmo.angular_diameter_distance(dd["redshift"])  # in Mpc
        
        # Convert all sizes to arcseconds
        r = np.array(
            np.maximum(
                (0.018 * u.Mpc / DA * u.rad).to(u.arcsec) / 0.168 / u.arcsec,
                70.0,
            ),
            dtype=float,
        )
        r = np.minimum(r, 500.0)
        dtype = np.dtype([("x", float), ("y",float), ("r", float)])
        xy_r = np.zeros(len(x), dtype=dtype)
        xy_r["x"] = x
        xy_r["y"] = y
        xy_r["r"] = r
        fitsio.write(out_fname, xy_r)
        del x, y, DA, r, xy_r, msk, patch_info, wcs, bbox
        gc.collect()
    if i % 100 ==0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600


/work/xiangchong.li/miniconda3/envs/image/lib/python3.11/site-packages/astropy/units/quantity.py:658: RuntimeWarning: divide by zero encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


1700
1800
1900
2000


In [ ]:
# Example array of physical sizes (in Mpc)
du = data
for i, entry in enumerate(full):
    tract_id = entry["tract"]
    patch_db = entry["patch"]
    patch_x = patch_db // 100
    patch_y = patch_db % 100
    patch_id = patch_x + patch_y * 9
    
    db_dir = "/lustre/work/xiangchong.li/work/hsc_s23b_data/catalogs/database/"
    outdir = f"{db_dir}/s23b-brightClusterMask/tracts/{tract_id}/{patch_id}"
    os.makedirs(outdir, exist_ok=True)
    out_fname = os.path.join(outdir, "catalog.fits")
    #if os.path.isfile(out_fname):
    #    continue
    patch_info = skymap[tract_id][patch_id]
    wcs = patch_info.getWcs()
    bbox = patch_info.getOuterBBox()
    poly_p = patch_info.getOuterSkyPolygon().getBoundingCircle()
    msk = poly_p.contains(du["ra"] / 180 * np.pi, du["dec"]/ 180 * np.pi)
    dd =  du[msk]
    if len(dd) > 0:
        x, y = wcs.skyToPixelArray(ra=dd["ra"], dec=dd["dec"], degrees=True)
        x = np.array(x - bbox.getBeginX(), dtype=float)
        y = np.array(y - bbox.getBeginY(), dtype=float)
        
        # Angular diameter distance at z
        DA = cosmo.angular_diameter_distance(dd["z_cl"])  # in Mpc
        
        # Convert all sizes to arcseconds
        r = np.array(
            np.maximum(
                (0.05 * u.Mpc / DA * u.rad).to(u.arcsec) / 0.168 / u.arcsec,
                70.0,
            ),
            dtype=float,
        )
        dtype = np.dtype([("x", float), ("y",float), ("r", float)])
        xy_r = np.zeros(len(x), dtype=dtype)
        xy_r["x"] = x
        xy_r["y"] = y
        xy_r["r"] = r
        fitsio.write(out_fname, xy_r)
        del x, y, DA, r, msk, patch_info, wcs, bbox
        gc.collect()
        print(i)

In [29]:
xy_r

array([(2757.13618426, 3041.62137113, 75.51352573)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('r', '<f8')])